# 1.Load and Prepare the Data

In [1]:
import os
import sys
import logging
import itertools
import multiprocessing
import numpy as np
import pandas as pd
import json
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.distributed as dist
import torch.multiprocessing as mp
import matplotlib.pyplot as plt

from itertools import islice
from tqdm import tqdm
from datetime import datetime
from collections import Counter
from concurrent.futures import ThreadPoolExecutor, as_completed
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import Dataset, DataLoader, TensorDataset
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import load_dataset
from keras.utils import to_categorical

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

'''
# dataset = load_dataset("mlcore/arxiv-classifier", name="major") # imbalanced
# dataset = load_dataset("mlcore/arxiv-classifier", name="all2023") # all articles from 2023 only
dataset = load_dataset("mlcore/arxiv-classifier", name="default") # minor (balanced)
minor_train = dataset['train']
train_categories = minor_train['primary_subfield']
minor_test = dataset['test']
test_categories = minor_test['primary_subfield']

# Extract features and labels
texts = [item['text'] for item in dataset]
labels = [item['label'] for item in dataset]  # Adjust the key as per the dataset's structure
# Split the dataset into training and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=1, shuffle=True)
'''

/Users/sro/2024ML-Research/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'\n# dataset = load_dataset("mlcore/arxiv-classifier", name="major") # imbalanced\n# dataset = load_dataset("mlcore/arxiv-classifier", name="all2023") # all articles from 2023 only\ndataset = load_dataset("mlcore/arxiv-classifier", name="default") # minor (balanced)\nminor_train = dataset[\'train\']\ntrain_categories = minor_train[\'primary_subfield\']\nminor_test = dataset[\'test\']\ntest_categories = minor_test[\'primary_subfield\']\n\n# Extract features and labels\ntexts = [item[\'text\'] for item in dataset]\nlabels = [item[\'label\'] for item in dataset]  # Adjust the key as per the dataset\'s structure\n# Split the dataset into training and testing sets\ntrain_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=1, shuffle=True)\n'

In [2]:
JSON_8 = "/Users/sro/2024ML-Research/Performance_Metrics/datasets/minor_train_processed.json"
JSON_9 = "/Users/sro/2024ML-Research/Performance_Metrics/datasets/minor_test_processed.json"
df_minor_train = pd.read_json(JSON_8, lines = "True")
df_minor_test = pd.read_json(JSON_9, lines = "True")
formatted_num_rows = "{:,}".format(len(df_minor_train))
print(f"Total number of minor train set rows: {formatted_num_rows}\n")
formatted_num_rows = "{:,}".format(len(df_minor_test))
print(f"Total number of minor test set rows: {formatted_num_rows}\n")

'''
df_minor = pd.concat([df_minor_train, df_minor_test], axis=0, ignore_index=True)
df_minor.to_json('/Users/sro/2024ML-Research/Performance_Metrics/datasets/minor.json', orient='records', lines=True)

JSON_10 = "/Users/sro/2024ML-Research/Performance_Metrics/datasets/minor_processed.json"
df_minor = pd.read_json(JSON_10, lines = "True")
formatted_num_rows = "{:,}".format(len(df_minor))
print(f"Total number of minor set rows: {formatted_num_rows}\n")
'''

Total number of minor train set rows: 108,696

Total number of minor test set rows: 27,178



'\ndf_minor = pd.concat([df_minor_train, df_minor_test], axis=0, ignore_index=True)\ndf_minor.to_json(\'/Users/sro/2024ML-Research/Performance_Metrics/datasets/minor.json\', orient=\'records\', lines=True)\n\nJSON_10 = "/Users/sro/2024ML-Research/Performance_Metrics/datasets/minor_processed.json"\ndf_minor = pd.read_json(JSON_10, lines = "True")\nformatted_num_rows = "{:,}".format(len(df_minor))\nprint(f"Total number of minor set rows: {formatted_num_rows}\n")\n'

In [3]:
# If splitting the DataFrame into training and test sets, reset the indices to ensure they are continuous and match correctly.
df_minor_train = df_minor_train.reset_index(drop=True)

categories = df_minor_train['primary_subfield']
primary_categories_train = [category.split()[0] for category in categories]
fulltexts_train = df_minor_train['fulltext']

titles_and_abstracts_train = []
abstracts = [sentence for sentence in df_minor_train['abstract']]
titles = [sentence for sentence in df_minor_train['title']]
for title, abstract in zip(titles, abstracts):
    combined = title + abstract
    titles_and_abstracts_train.append(combined)

label_encoder = LabelEncoder()
label_encoder.fit(primary_categories_train)
y_train_integer = label_encoder.transform(primary_categories_train)
num_categories = len(label_encoder.classes_)
#print(label_encoder.classes_)
# y_one_hot = to_categorical(y_integer, num_classes = num_categories)

categories = df_minor_test['primary_subfield']
primary_categories_test = [category.split()[0] for category in categories]
y_test_integer = label_encoder.transform(primary_categories_test)
fulltexts_test = df_minor_test['fulltext']

titles_and_abstracts_test = []
abstracts = [sentence for sentence in df_minor_test['abstract']]
titles = [sentence for sentence in df_minor_test['title']]
for title, abstract in zip(titles, abstracts):
    combined = title + abstract
    titles_and_abstracts_test.append(combined)


In [4]:
print(label_encoder.classes_)

['astro-ph' 'astro-ph.CO' 'astro-ph.EP' 'astro-ph.GA' 'astro-ph.HE'
 'astro-ph.IM' 'astro-ph.SR' 'cond-mat.dis-nn' 'cond-mat.mes-hall'
 'cond-mat.mtrl-sci' 'cond-mat.other' 'cond-mat.quant-gas' 'cond-mat.soft'
 'cond-mat.stat-mech' 'cond-mat.str-el' 'cond-mat.supr-con' 'cs.AI'
 'cs.AR' 'cs.CC' 'cs.CE' 'cs.CG' 'cs.CL' 'cs.CR' 'cs.CV' 'cs.CY' 'cs.DB'
 'cs.DC' 'cs.DL' 'cs.DM' 'cs.DS' 'cs.ET' 'cs.FL' 'cs.GL' 'cs.GR' 'cs.GT'
 'cs.HC' 'cs.IR' 'cs.IT' 'cs.LG' 'cs.LO' 'cs.MA' 'cs.MM' 'cs.MS' 'cs.NE'
 'cs.NI' 'cs.OH' 'cs.OS' 'cs.PF' 'cs.PL' 'cs.RO' 'cs.SC' 'cs.SD' 'cs.SE'
 'cs.SI' 'econ.EM' 'econ.TH' 'eess.AS' 'eess.IV' 'eess.SP' 'eess.SY'
 'gr-qc' 'hep-ex' 'hep-lat' 'hep-ph' 'hep-th' 'math-ph' 'math.AC'
 'math.AG' 'math.AP' 'math.AT' 'math.CA' 'math.CO' 'math.CT' 'math.CV'
 'math.DG' 'math.DS' 'math.FA' 'math.GN' 'math.GR' 'math.GT' 'math.HO'
 'math.KT' 'math.LO' 'math.MG' 'math.NA' 'math.NT' 'math.OA' 'math.OC'
 'math.PR' 'math.QA' 'math.RA' 'math.RT' 'math.SG' 'math.SP' 'math.ST'
 'nlin.AO' 

In [ ]:
X_train_1, X_eval_1, y_train_1, y_eval_1 = train_test_split(fulltexts_train, y_train_integer, test_size=0.2, random_state=42)

train_data = pd.DataFrame({'fulltext': fulltexts_train, 'title_abstract': titles_and_abstracts_train, 'label': y_train_integer})
test_data = pd.DataFrame({'fulltext': fulltexts_test, 'title_abstract': titles_and_abstracts_test, 'label': y_test_integer})

train_data.to_json('/Users/sro/2024ML-Research/MPNet/minor_train.json', orient='records', lines=True)
test_data.to_json('/Users/sro/2024ML-Research/MPNet/minor_test.json', orient='records', lines=True)

print("Train and test data saved as JSON files")

In [5]:
class ArxivDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        try:
            text = self.texts[idx]
            label = self.labels[idx]
        except KeyError as e:
            print(f"KeyError: {e}. Check if index {idx} is valid.")
            raise
        
        encoding = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length, 
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)
        }

JSON_1 = '/Users/sro/2024ML-Research/MPNet/minor_train.json'
JSON_2 = '/Users/sro/2024ML-Research/MPNet/minor_test.json'
df_minor_train = pd.read_json(JSON_1, lines = "True")
df_minor_test = pd.read_json(JSON_2, lines = "True")
X_train = df_minor_train['fulltext'] # could use title + abstract for vector inference instead of full text
y_train = df_minor_train['label']
num_categories = len(set(y_train))
X_test = df_minor_test['fulltext'] # could use title + abstract for vector inference instead of full text
y_test = df_minor_test['label']

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')

train_dataset = ArxivDataset(X_train, y_train, tokenizer, max_length=512) #  a maximum sequence length of tokens as context window is set here
test_dataset = ArxivDataset(X_test, y_test, tokenizer, max_length=512) # 512 is usally by default

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=DataCollatorWithPadding(tokenizer))
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, collate_fn=DataCollatorWithPadding(tokenizer))

#DataCollatorWithPadding for dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [20]:
test_dataset[0]

{'input_ids': tensor([    0, 16857,  2968,  1015, 10576,  5298, 16105,  6309, 29241,  2002,
         24313, 18423,  2003,  2000,  3306,  2114,  2001, 13679,  1015,  2083,
          5673, 12174,  2479,  3370,  2513,  1015, 12174, 24318, 24313, 16025,
         20354,  2019,  1043,  1016,  1015,  1043,  1016,  8806,  2491,  1014,
          1020,  1014,  1053,  1016,  1052,  1016,  5576,  9081,  2491,  1014,
          1020,  1014,  1052,  1016,  2006,  2513,  1014,  1064,  1016, 12853,
          2230,  2513,  1014,  1053,  1016, 11378,  2513,  1014,  1063,  1016,
          5474,  2491,  1014,  1054,  1016, 19137,  2491,  1014,  1051,  1016,
          1015,  1052,  1016,  7422,  2513,  2002,  1054,  1016,  1015,  1043,
          1016,  6304,  2236,  2491,  1014,  1020,  1014,  1022,  1012,  1019,
          2537,  2001,  5588,  1014,  2666,  2824,  2001,  2978,  1014, 18884,
          1014,  6191, 19993, 17792,  1014,  3919,  1020,  2824,  2009,  8563,
          2596,  2002,  3047,  1014,  2

# 2.Fine-Tune the MPNet Model

training a classification head and update some weights of MPNet model

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc}

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') cuda does not work on Mac
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

model = AutoModelForSequenceClassification.from_pretrained('sentence-transformers/all-mpnet-base-v2', num_labels=num_categories)

# recommended -> default option, i.e., the weights of both the pretrained model and the classification head will be updated

# Freeze all layers of the pretrained model and only the classification head will be fine-tuned
#for param in model.base_model.parameters():
#   param.requires_grad = False

model.to(device)

# Set up training arguments
training_args = TrainingArguments(
    output_dir='/Users/sro/2024ML-Research/MPNet/results',
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='/Users/sro/2024ML-Research/MPNet/logs',
    logging_steps=100,  
    eval_strategy="epoch", 
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    #dataloader_num_workers=4
    #fp16=True #fp16 mixed precision requires a GPU (not 'mps').
)
    
# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

model.save_pretrained('/Users/sro/2024ML-Research/MPNet/fine_tuned_mpnet/MPNet_model_fine_tuned') # save the enitre model including the new head
tokenizer.save_pretrained('/Users/sro/2024ML-Research/MPNet/fine_tuned_mpnet/MPNet_tokenizer')

Using MPNet alone to make predictions on new data

In [7]:
tokenizer = AutoTokenizer.from_pretrained('/Users/sro/2024ML-Research/MPNet/fine_tuned_mpnet/MPNet_tokenizer_6_epochs_minor')
model = AutoModelForSequenceClassification.from_pretrained('/Users/sro/2024ML-Research/MPNet/fine_tuned_mpnet/MPNet_model_fine_tuned_6_epochs_minor')
# Must load AutoModelForSequenceClassification instead of AutoModel to put back the classification head we have trained!!

# Move model to the appropriate device
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
model.to(device)

print(f"Using device: {device}")

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc}

# Set up training arguments
training_args = TrainingArguments(
    output_dir='/Users/sro/2024ML-Research/MPNet/results',
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='/Users/sro/2024ML-Research/MPNet/logs',
    logging_steps=100,  
    eval_strategy="epoch", 
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

results = trainer.evaluate(eval_dataset=test_dataset)

print(f"Test Accuracy: {results['eval_accuracy']:.4f}")

Using device: mps


Test Accuracy: 0.7247


# 3. Compute Embeddings Using the Fine-Tuned Model

cell below contains alternative workflow

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('/Users/sro/2024ML-Research/MPNet/fine_tuned_mpnet/MPNet_tokenizer_6_epochs_minor')
model = AutoModel.from_pretrained('/Users/sro/2024ML-Research/MPNet/fine_tuned_mpnet/MPNet_model_fine_tuned_6_epochs_minor')
#for embeddings: Use AutoModel
#for classification tasks: Use AutoModelForSequenceClassification

#ensure the model is in evaluation mode
model.eval()

device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

# Use DataParallel for multi-GPU processing
#if torch.cuda.device_count() > 1:
#    print(f"Using {torch.cuda.device_count()} GPUs for processing")
#    model = torch.nn.DataParallel(model)

model.to(device)

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(
        token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(
            input_mask_expanded.sum(1), min=1e-9)

def batched(iterable, n):
    it = iter(iterable)
    while (batch := list(islice(it, n))):
        yield batch
        
#function to compute embeddings
def compute_embeddings(texts, batch_size=16):
    dataloader = DataLoader(texts, batch_size=batch_size, shuffle=False)
    embeddings = []
    with torch.no_grad():
        for batch in tqdm(dataloader):
            encoded_input = tokenizer(batch, padding=True, truncation=True, return_tensors='pt')
            encoded_input = {key: val.to(device) for key, val in encoded_input.items()}
            model_output = model(**encoded_input)
            mean_pooled = mean_pooling(model_output, encoded_input['attention_mask'])
            normalized_embeddings = F.normalize(mean_pooled, p=2, dim=1)
            embeddings.append(normalized_embeddings.cpu().numpy())
    return np.concatenate(embeddings, axis=0)

train_embeddings = compute_embeddings(X_train)
test_embeddings = compute_embeddings(X_test)

np.save('/Users/sro/2024ML-Research/MPNet/MPNet_train_embeddings_fulltext_6epochs_minor.npy', train_embeddings)
np.save('/Users/sro/2024ML-Research/MPNet/MPNet_test_embeddings_fulltext_6epochs_minor.npy', test_embeddings)

print("Embeddings computed successfully!")

# 4. Loading the Embeddings and Preparing the Data

In [9]:
X_train_embeddings = np.load('/Users/sro/2024ML-Research/MPNet/MPNet_train_embeddings_fulltext_6epochs_minor.npy')
X_test_embeddings = np.load('/Users/sro/2024ML-Research/MPNet/MPNet_test_embeddings_fulltext_6epochs_minor.npy')

# y_train_one_hot = to_categorical(y_train, num_classes = num_categories)
# y_test_one_hot = to_categorical(y_test, num_classes = num_categories)

#convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_embeddings, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_embeddings, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

#create DataLoader for training
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# 5. Defining the Artificial Neural Network Model

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_prob=0.5):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_prob) 
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.dropout(out)  # Apply dropout
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.dropout(out)  # Apply dropout
        out = self.fc3(out)
        return out

# Hyperparameters to fine-tune
hidden_dims = [512, 1024, 2048]
num_epochs_list = [20, 30, 40]
dropout_prob = 0.5
input_dim = X_train_embeddings.shape[1]
output_dim = len(label_encoder.classes_)

# Function to train and evaluate the model
def train_and_evaluate_model(hidden_dim, num_epochs):
    # Initialize the model
    model = SimpleNN(input_dim, hidden_dim, output_dim, dropout_prob)
    
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)

    # Store accuracy values
    accuracy_values = []

    # Training the model
    for epoch in range(num_epochs):
        model.train()
        for batch_x, batch_y in train_loader:
            # Forward pass
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
        
        # Evaluate the model
        model.eval()
        with torch.no_grad():
            outputs = model(X_test_tensor)
            _, predicted = torch.max(outputs.data, 1)
            y_hat = predicted.cpu().numpy()
            y_true = y_test_tensor.cpu().numpy()
            accuracy = accuracy_score(y_true, y_hat)
            accuracy_values.append(accuracy)
            print(f'Epoch [{epoch+1}/{num_epochs}], Test Accuracy: {accuracy:.4f}')

    # Print classification report
    print(classification_report(y_true, y_hat, target_names=label_encoder.classes_))

    return accuracy_values

# Perform grid search and plot accuracy
results = {}
for hidden_dim, num_epochs in itertools.product(hidden_dims, num_epochs_list):
    print(f'Training with Hidden Dim: {hidden_dim}, Epochs: {num_epochs}')
    accuracy_values = train_and_evaluate_model(hidden_dim, num_epochs)
    results[(hidden_dim, num_epochs)] = accuracy_values

# Plot accuracy vs. epochs for each hidden dimension
for (hidden_dim, num_epochs), accuracies in results.items():
    plt.plot(range(1, num_epochs + 1), accuracies, label=f'Hidden Dim: {hidden_dim}, Epochs: {num_epochs}')

plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy vs. Epochs')
plt.legend()
plt.show()

# 6. Training the ANN Model

In [ ]:
hidden_dim = 512
dropout_prob = 0.5
input_dim = X_train_embeddings.shape[1]
output_dim = len(label_encoder.classes_)

class SimpleNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_prob=0.5):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_prob) 
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.dropout(out)  # Apply dropout
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.dropout(out)  # Apply dropout
        out = self.fc3(out)
        return out
    
model = SimpleNN(input_dim, hidden_dim, output_dim, dropout_prob)
    
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training the model
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    for batch_x, batch_y in train_loader:
        # Forward pass
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    

# 7. Evaluating the Model

In [ ]:
model.eval()

with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs.data, 1)
    y_hat = predicted.cpu().numpy()
    y_true = y_test_tensor.cpu().numpy()
    accuracy = accuracy_score(y_true, y_hat)
    print(f'Test Accuracy: {accuracy:.4f}')

print(classification_report(y_true, y_hat, target_names=label_encoder.classes_))